In [ ]:
from reader import readShortVideo
from reader import getVideoList
import matplotlib.pyplot as plt
from os import listdir
import os
import pandas as pd
import numpy as np
import pickle

import torchvision
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
%matplotlib inline
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
# load data
class CustomDataset(Dataset):
    def __init__(self, train_valid="train"):
        if train_valid == "train":
            with open("../train_X.pkl", "rb") as f:
                self.X = pickle.load(f)
            with open("../train_y.pkl", "rb") as f:
                self.y = pickle.load(f)
                
        if train_valid == "valid":
            with open("../valid_X.pkl", "rb") as f:
                self.X = pickle.load(f)
            with open("../valid_y.pkl", "rb") as f:
                self.y = pickle.load(f)
    def __getitem__(self, index):
        single_image = self.X[index]
        single_label = self.y[index]

        # Return image and the label
        return single_image, single_label

    def __len__(self):
        return self.length

train_dataset = CustomDataset("train")
valid_dataset = CustomDataset("valid")

In [ ]:
cnn_feature_extractor = torchvision.models.densenet121(pretrained=True).features.cuda() # to 1000 dims

In [ ]:
with torch.no_grad():
    output_size = cnn_feature_extractor(train_dataset[0][0].cuda()).size()
print(output_size)

In [ ]:
# create sequential features for RNN
feature_size = 1024*7*7
cnn_feature_extractor.eval()
train_features = []
counter = 0
with torch.no_grad():
    for i in range(len(train_dataset.X)):
        input_X = train_dataset[i][0]
        feature = cnn_feature_extractor(input_X.cuda()).cpu().view(-1, feature_size)
        train_features.append(feature)
        counter +=1
        if counter % 300 == 0:
            print(counter)
print("training instances done")

valid_features = []
counter = 0
with torch.no_grad():
    for i in range(len(valid_dataset.X)):
        input_X = valid_dataset[i][0]
        feature = cnn_feature_extractor(input_X.cuda()).cpu().view(-1, feature_size)
        valid_features.append(feature)
        counter +=1
        if counter % 100 == 0:
            print(counter)

In [ ]:
with open("../train_features_d12.pkl", "wb") as f:
    pickle.dump(train_features, f)
with open("../valid_features_d12.pkl", "wb") as f:
    pickle.dump(valid_features, f)
